In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Libraries

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
import seaborn as sns
from scipy import stats
from scipy.stats import norm, skew
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import  RobustScaler
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import RobustScaler, PolynomialFeatures
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error,make_scorer
import xgboost as xgb
import lightgbm as lgb
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
pip install mlxtend
from mlxtend.regressor import StackingCVRegressor
from sklearn.metrics import mean_squared_log_error

**Download the data**

In [ ]:
df_train = pd.read_csv('/kaggle/input/ml-competition-2024-for-ukrainians/train.csv')
df_test = pd.read_csv('/kaggle/input/ml-competition-2024-for-ukrainians/test.csv')
sample_submission = pd.read_csv('/kaggle/input/ml-competition-2024-for-ukrainians/sample_submission.csv')


In [ ]:
df_train

**What features we have**

1.Item_Identifier: A unique identifier for each product.

2.Item_Weight: The weight of the product.

3.Item_Fat_Content: The fat content of the product, which typically has values like 'Low Fat', 'Regular', etc.

4.Item_Visibility: The percentage of total display area allocated to this product in the store.

5.Item_Type: The category to which the product belongs, such as 'Dairy', 'Soft Drinks', 'Meat', etc.

6.Item_MRP: The maximum retail price (list price) of the product.

7.Outlet_Identifier: A unique identifier for each store.

8.Outlet_Establishment_Year: The year in which the store was established.

9.Outlet_Size: The size of the store in terms of ground area, usually categorized as 'Small', 'Medium', 'High'.

10.Outlet_Location_Type: The type of city in which the store is located, such as 'Tier 1', 'Tier 2', 'Tier 3'.

11.Outlet_Type: The type of store, such as 'Grocery Store', 'Supermarket Type1', 'Supermarket Type2', 'Supermarket Type3'.

12.Item_Outlet_Sales: The sales of the product in the particular store (target variable).

In [ ]:
sample_submission

In [ ]:
df_train.columns

**Lets drop "id" column from our test and train data set**

In [ ]:
#check the numbers of samples and features
print("The train data size before dropping Id feature is : {} ".format(df_train.shape))
print("The test data size before dropping Id feature is : {} ".format(df_test.shape))

#Save the 'Id' column
train_ID = df_train['id']
test_ID = df_test['id']

#Now drop the  'Id' colum since it's unnecessary for  the prediction process.
df_train.drop("id", axis = 1, inplace = True)
df_test.drop("id", axis = 1, inplace = True)

#check again the data size after dropping the 'Id' variable
print("\nThe train data size after dropping Id feature is : {} ".format(df_train.shape)) 
print("The test data size after dropping Id feature is : {} ".format(df_test.shape))

In [ ]:
df_train.shape


# EDA ( Exploratory Data Analysis)

**Lets look to distribution of our target - Item_Outlet_Sales**

In [ ]:
sns.histplot(df_train['Item_Outlet_Sales'])

In [ ]:
fig, ax = plt.subplots()
ax.scatter(x = df_train['Item_MRP'], y = df_train['Item_Outlet_Sales'])
plt.ylabel('Item_Outlet_Sales', fontsize=13)
plt.xlabel('Item_Weight', fontsize=13)
plt.show()

In [ ]:
df_train['Item_Outlet_Sales'].describe()

# **1. Deviate from the Normal Distribution**

When we say that the distribution of Item_Outlet_Sales deviates from the normal distribution, it means that the shape of its distribution does not follow the familiar bell curve of a normal distribution. This can have several implications:

Asymmetry: The distribution might be skewed to the left or right rather than being symmetric.
Kurtosis: The distribution might have more or less peakedness compared to a normal distribution.
Outliers: There could be more outliers than expected in a normal distribution.
Understanding this deviation is crucial because many statistical methods and machine learning algorithms assume normality. If the data deviates significantly from normality, we might need to apply transformations (like logarithmic, square root, etc.) or use non-parametric methods.

# **2. Appreciable Positive Skewness**
Positive skewness (right-skewed) means that the tail on the right side of the distribution is longer or fatter than the left side. For Item_Outlet_Sales, this implies:

Higher Sales Outliers: There are some stores with much higher sales than the majority.
Mean > Median: The mean of the sales will be greater than the median because of the high-value outliers.
Implications of positive skewness include:

Model Performance: Some models, especially linear ones, may not perform well if the target variable is highly skewed.
Interpretation: Statistical measures like the mean can be misleading.
# **3. Show Peakedness (Kurtosis)**
Peakedness, or kurtosis, refers to the "tailedness" of the distribution. For Item_Outlet_Sales:

High Kurtosis (Leptokurtic): This would mean a sharper peak with fatter tails, indicating more data in the tails and fewer in the shoulders.
Low Kurtosis (Platykurtic): This indicates a flatter peak with thinner tails.
Implications of kurtosis include:

Risk Assessment: Higher kurtosis might indicate more risk in terms of prediction errors since extreme values (outliers) are more common.
Distribution Shape: It helps in understanding the concentration of data points around the mean.


In [ ]:
print('Skewness: %f' % df_train['Item_Outlet_Sales'].skew())
print('Kurtsis: %f' %df_train['Item_Outlet_Sales'].kurt())

# Interpretation of Skewness and Kurtosis Values
**Skewness:**
0: Perfectly symmetrical.

more 0: Positively skewed (right-skewed).

<0: Negatively skewed (left-skewed).

**Kurtosis:**
0: Mesokurtic (normal distribution).

more 0: Leptokurtic (sharper peak, fatter tails).

<0: Platykurtic (flatter peak, thinner tails).

Skewness: 3.073422
This is a high positive skewness value. Positive skewness means that the right tail of the distribution is longer or fatter than the left. In practical terms:

Most Sales Are Low to Moderate: Most store sales are clustered at lower values, with fewer instances of extremely high sales.
Outliers: There are a few instances where sales are significantly higher than the majority of the data.
Mean > Median: The mean sales value will be higher than the median due to these few high sales values.
Kurtosis: 28.001507
This is a very high kurtosis value. High kurtosis (leptokurtic distribution) means that the distribution has a sharp peak and fat tails. In practical terms:

Sharp Peak: Most sales values are very close to the mean value.
Fat Tails: There are more extreme values (outliers) than you would expect in a normal distribution.
Risk: This indicates higher risk, as extreme values are more likely to occur, which can affect the stability and performance of your model.
Implications for Modeling
Given these characteristics, here's what you might consider doing for better model performance:

Transformations to Normalize Data:

Log Transformation: Applying a log transformation can help in reducing skewness.
Square Root Transformation: This can also help in reducing skewness but is less powerful than log transformation.
Box-Cox Transformation: This can handle both skewness and stabilize variance.
Handling Outliers:

Cap and Floor: You can cap the extreme values at a certain threshold to reduce their impact.
Outlier Removal: In some cases, removing extreme outliers can help.
Robust Models:

Tree-Based Models: Models like Random Forests, Gradient Boosting Machines (GBM), and XGBoost are less sensitive to skewed distributions and outliers.
Regularization: Using regularized models like Ridge or Lasso regression can help in dealing with skewed data.

# Log-transformation of the target variable

In [ ]:
#We use the numpy fuction log1p which  applies log(1+x) to all elements of the column
df_train["Item_Outlet_Sales"] = np.log1p(df_train["Item_Outlet_Sales"])

#Check the new distribution 
sns.distplot(df_train['Item_Outlet_Sales'] , fit=norm);

# Get the fitted parameters used by the function
(mu, sigma) = norm.fit(df_train['Item_Outlet_Sales'])
print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))

#Now plot the distribution
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')
plt.ylabel('Frequency')
plt.title('Sales distribution')

#Get also the QQ-plot
fig = plt.figure()
res = stats.probplot(df_train['Item_Outlet_Sales'], plot=plt)
plt.show()

# Finding missing values

In [ ]:
df_train.apply(lambda x: sum(x.isnull()))

In [ ]:
df_train.apply(lambda x : len(x.unique()))

This tells us that there are 1559 products and 10 outlets/stores and no missing values

**Lets look at boxplots for different features**

In [ ]:
plt.figure(figsize = (10,9))

plt.subplot(311)
sns.boxplot(x='Outlet_Size', y='Item_Outlet_Sales', data=df_train, palette="Set1")

plt.subplot(312)
sns.boxplot(x='Outlet_Location_Type', y='Item_Outlet_Sales', data=df_train, palette="Set1")

plt.subplot(313)
sns.boxplot(x='Outlet_Type', y='Item_Outlet_Sales', data=df_train, palette="Set1")

plt.subplots_adjust(wspace = 0.2, hspace = 0.4,top = 1.5)

plt.show()

In [ ]:
plt.figure(figsize = (14,9))

plt.subplot(211)
ax = sns.boxplot(x='Outlet_Identifier', y='Item_Outlet_Sales', data=df_train, palette="Set1")
ax.set_title("Outlet_Identifier vs. Item_Outlet_Sales", fontsize=15)
ax.set_xlabel("", fontsize=12)
ax.set_ylabel("Item_Outlet_Sales", fontsize=12)

plt.subplot(212)
ax = sns.boxplot(x='Item_Type', y='Item_Outlet_Sales', data=df_train, palette="Set1")
ax.set_title("Item_Type vs. Item_Outlet_Sales", fontsize=15)
ax.set_xlabel("", fontsize=12)
ax.set_ylabel("Item_Outlet_Sales", fontsize=12)

plt.subplots_adjust(hspace = 0.9, top = 0.9)
plt.setp(ax.get_xticklabels(), rotation=45)

plt.show()

In [ ]:
df_train['Item_Visibility'].describe()

**There is the problem. Item_visibility can not be zero.**

# Preprocessing steps


Drop some columns that are not important. ( I experimented with different columns)

In [ ]:
df_train.drop(["Outlet_Identifier", "Outlet_Establishment_Year"],axis=1,inplace=True)
df_test.drop(["Outlet_Identifier", "Outlet_Establishment_Year"],axis=1,inplace=True)

Fill missing values in Item_Visibility with mean value.

In [ ]:
#Average visibility of the product
visibility_avg = df_train.pivot_table(values = 'Item_Visibility', index='Item_Identifier')

#Impute 0 values with mean visibility of that product:
missing_values = (df_train['Item_Visibility'] ==0)

print ('Number of 0 values initially: %d'%sum(missing_values))
df_train.loc[missing_values,'Item_Visibility'] = df_train.loc[missing_values,'Item_Identifier'].apply(lambda x: visibility_avg.at[x, 'Item_Visibility'])
print ('Number of 0 values after modification: %d'%sum(df_train['Item_Visibility'] == 0))

In [ ]:
#Average visibility of the product
visibility_avg = df_test.pivot_table(values = 'Item_Visibility', index='Item_Identifier')

#Impute 0 values with mean visibility of that product:
missing_values = (df_test['Item_Visibility'] ==0)

print ('Number of 0 values initially: %d'%sum(missing_values))
df_test.loc[missing_values,'Item_Visibility'] = df_test.loc[missing_values,'Item_Identifier'].apply(lambda x: visibility_avg.at[x, 'Item_Visibility'])
print ('Number of 0 values after modification: %d'%sum(df_test['Item_Visibility'] == 0))

In this section, we will clean the `Item_Fat_Content` column by replacing inconsistent category names with more uniform ones. This helps in ensuring that the data is consistent and ready for further analysis or modeling.

In [ ]:
print('Original Categories:')
print(df_train['Item_Fat_Content'].value_counts())

print('\nModified Categories:')
df_train['Item_Fat_Content'] = df_train['Item_Fat_Content'].replace({'LF':'Low Fat',
                                                             'reg':'Regular',
                                                             'low fat':'Low Fat'})
print(df_train['Item_Fat_Content'].value_counts())

In [ ]:
print('Original Categories:')
print(df_test['Item_Fat_Content'].value_counts())

print('\nModified Categories:')
df_test['Item_Fat_Content'] = df_test['Item_Fat_Content'].replace({'LF':'Low Fat',
                                                             'reg':'Regular',
                                                             'low fat':'Low Fat'})
print(df_test['Item_Fat_Content'].value_counts())

In this section, we will encode the `Outlet_Size` and `Outlet_Location_Type` column by mapping its categorical values to numerical values. This transformation helps in converting categorical data into a format that can be used for machine learning algorithms.

In [ ]:
df_train['Outlet_Size'] = df_train['Outlet_Size'].map({'Small'  : 1,
                                                 'Medium' : 2,
                                                 'High'   : 3
                                                 }).astype(int)

df_test['Outlet_Size'] = df_test['Outlet_Size'].map({'Small'  : 1,
                                               'Medium' : 2,
                                               'High'   : 3
                                              }).astype(int)

In [ ]:
# Outlet_Location_Type feature encoding by getting the last character and converting to int type

df_train['Outlet_Location_Type'] = df_train['Outlet_Location_Type'].map({'Tier 1': 1,
                                                                         'Tier 2': 2,
                                                                         'Tier 3': 3
                                                                        }).astype(int)
df_test['Outlet_Location_Type'] = df_test['Outlet_Location_Type'].map({'Tier 1': 1,
                                                                        'Tier 2': 2,
                                                                        'Tier 3': 3
                                                                        }).astype(int)


In [ ]:
df_train['Item_Identifier']

In the Item_Type feature, there are 16 categories but when we look closely to Item_Identifier_Categories, it has first two characters defining the item type, these are:


**FD for probably Food;**

**DR for probably Drinks;**

**NC for probably Non-Consumables.**


So we'll drop the Item_Identifier feature and create a new column containing these categories.

In [ ]:
df_train['Item_Identifier_Categories'] = df_train['Item_Identifier'].str[0:2] #.astype(int)
df_test['Item_Identifier_Categories']  = df_test['Item_Identifier'].str[0:2]

sns.countplot(x=df_train['Item_Identifier_Categories'])

In [ ]:
# Let's drop useless columns
df_train.drop(labels=['Item_Identifier'], axis=1, inplace=True)
df_test.drop(labels=['Item_Identifier'],  axis=1, inplace=True)

# Split dataset to train and valid

In [ ]:
X = df_train.drop('Item_Outlet_Sales', axis =1)
y = df_train['Item_Outlet_Sales']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.05, random_state =42)

print(f"Training features shape: {X_train.shape}")
print(f"Validation features shape: {X_valid.shape}")
print(f"Training target shape: {y_train.shape}")
print(f"Validation target shape: {y_valid.shape}")

# Numerical and One-Hot Encoding of Categorical variables

In [ ]:
# Get list of categorical variables
s = (df_train.dtypes == 'object')
object_cols = list(s[s].index)

print("Categorical variables:")
print(object_cols)


In [ ]:
onehot_encoder = OneHotEncoder(sparse = False)

X_train_encoded = pd.DataFrame(onehot_encoder.fit_transform(X_train[object_cols]))

X_valid_encoded = pd.DataFrame(onehot_encoder.transform(X_valid[object_cols]))

X_test_encoded = pd.DataFrame(onehot_encoder.transform(df_test[object_cols]))

# Add back the encoded column names
X_train_encoded.columns = onehot_encoder.get_feature_names_out(object_cols)
X_valid_encoded.columns = onehot_encoder.get_feature_names_out(object_cols)
X_test_encoded.columns = onehot_encoder.get_feature_names_out(object_cols)

# Reset index to avoid issues during concatenation
X_train_encoded.reset_index(drop=True, inplace=True)
X_valid_encoded.reset_index(drop=True, inplace=True)
X_test_encoded.reset_index(drop=True, inplace=True)
X_train.reset_index(drop=True, inplace=True)
X_valid.reset_index(drop=True, inplace=True)
df_test.reset_index(drop=True, inplace=True)

# Drop original categorical columns from X_train and X_valid
X_train = X_train.drop(object_cols, axis=1)
X_valid = X_valid.drop(object_cols, axis=1)
df_test = df_test.drop(object_cols, axis=1)

# Concatenate the encoded columns back to the original dataframes
X_train = pd.concat([X_train, X_train_encoded], axis=1)
X_valid = pd.concat([X_valid, X_valid_encoded], axis=1)
X_test = pd.concat([df_test, X_test_encoded], axis=1)

print(f"Encoded training features shape: {X_train.shape}")
print(f"Encoded validation features shape: {X_valid.shape}")
print(f"Encoded test features shape: {X_test.shape}")

**scalar**

In [ ]:


numerical_cols = ['Item_Weight', 'Item_Visibility', 'Item_MRP']

# Separate the numerical columns from the rest of the features
X_train_num = X_train[numerical_cols]
X_valid_num = X_valid[numerical_cols]
X_test_num = X_test[numerical_cols]

# Initialize the scaler
scaler = RobustScaler()

# Fit the scaler on the training data and transform the training, validation, and test data
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train_num), columns=numerical_cols)
X_valid_scaled = pd.DataFrame(scaler.transform(X_valid_num), columns=numerical_cols)
X_test_scaled = pd.DataFrame(scaler.transform(X_test_num), columns=numerical_cols)

# Drop the original numerical columns from the datasets
X_train.drop(numerical_cols, axis=1, inplace=True)
X_valid.drop(numerical_cols, axis=1, inplace=True)
X_test.drop(numerical_cols, axis=1, inplace=True)

# Concatenate the scaled numerical columns back to the datasets
X_train = pd.concat([X_train.reset_index(drop=True), X_train_scaled.reset_index(drop=True)], axis=1)
X_valid = pd.concat([X_valid.reset_index(drop=True), X_valid_scaled.reset_index(drop=True)], axis=1)
X_test = pd.concat([X_test.reset_index(drop=True), X_test_scaled.reset_index(drop=True)], axis=1)

print(f"Scaled training features shape: {X_train.shape}")
print(f"Scaled validation features shape: {X_valid.shape}")
print(f"Scaled test features shape: {X_test.shape}")

In [ ]:
# Replacing spaces in column names with underscores
X_train.columns = [col.replace(' ', '_') for col in X_train.columns]
X_valid.columns = [col.replace(' ', '_') for col in X_valid.columns]
X_test.columns = [col.replace(' ', '_') for col in X_test.columns]

# Model. Training

In [ ]:
#Metric
def rmsle(y_true, y_pred):
    return np.sqrt(mean_squared_log_error(y_true, y_pred))



# Define base models ( All parameters were found using GridSearchCV)
lasso = Lasso(alpha=0.0001, fit_intercept = True)
randomforest = RandomForestRegressor(n_estimators = 200, min_samples_leaf = 100, max_depth = None, n_jobs = -1, random_state = 42)
xgboost = XGBRegressor(learning_rate=0.1, max_depth=5, n_estimators=100)
lightgbm = LGBMRegressor(
    learning_rate=0.05,
    max_depth=30,
    n_estimators=1000,
    num_leaves=100,
    min_child_samples=50,
    metric='rmsle',
    boosting_type='gbdt',
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=0.1,
    reg_lambda=0.1,
    random_state=42
)
ridge = Ridge(alpha=3, fit_intercept = True, solver = 'sparse_cg')
# Define meta-model
meta_model = XGBRegressor(learning_rate=0.1, max_depth=5, n_estimators=100)

# Initialize the StackingCVRegressor
stack_gen = StackingCVRegressor(regressors=(ridge,lasso,randomforest, xgboost, lightgbm),
                                meta_regressor=meta_model,
                                use_features_in_secondary=True,
                                cv=5)

stack_gen.fit(X_train, y_train)


y_train_pred = stack_gen.predict(X_train)
y_valid_pred = stack_gen.predict(X_valid)
y_test_pred = stack_gen.predict(X_test)

# Ensure predictions are non-negative
y_train_pred = np.maximum(y_train_pred, 0)
y_valid_pred = np.maximum(y_valid_pred, 0)
y_test_pred = np.maximum(y_test_pred, 0)

# Inverse transformation of predictions to the original scale
y_train_pred_original = np.expm1(y_train_pred)
y_valid_pred_original = np.expm1(y_valid_pred)

train_rmsle = rmsle(np.expm1(y_train), y_train_pred_original)
valid_rmsle = rmsle(np.expm1(y_valid), y_valid_pred_original)

print(f'Training RMSLE: {train_rmsle}')
print(f'Validation RMSLE: {valid_rmsle}')

# Submission

In [ ]:
y_test_pred_original_scale = np.expm1(y_test_pred)
sample_submission['Item_Outlet_Sales'] = y_test_pred_original_scale
submission = sample_submission
submission.to_csv('submission.csv', index = False)